<a href="https://colab.research.google.com/github/Ok3ks/Disso-COLD/blob/main/BERT_20newsgroup_HAN_Imp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install transformers 
%pip install sklearn
%pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments,DataCollatorWithPadding
from datasets import load_dataset, load_metric,Dataset
import torch
import numpy as np
from utils import AssessData,RoBERT_Model_2
from datasets import load_dataset
from utils import PrepareCorpus,AssessData
from sklearn.model_selection import train_test_split
from datasets import Dataset,load_dataset
import json
from re import template
from sklearn.model_selection import KFold

20 NEWSGROUP

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
training_dir = "/content/drive/MyDrive/ML/Dissertation/training_cache"

In [5]:
parentdir = "/content/drive/MyDrive/ML/Datasets/20News"

train =parentdir+"/20news-bydate"+"/20news-bydate-train"
test = parentdir+"/20news-bydate"+ "/20news-bydate-test"

In [6]:
id2labels = {'talk.politics.mideast': 0, 'rec.autos': 1, 'comp.sys.mac.hardware': 2, 'alt.atheism': 3, 'rec.sport.baseball': 4, 'comp.os.ms-windows.misc': 5, 'rec.sport.hockey': 6, 'sci.crypt': 7, 'sci.med': 8, 'talk.politics.misc': 9, 'rec.motorcycles': 10, 'comp.windows.x': 11, 'comp.graphics': 12, 'comp.sys.ibm.pc.hardware': 13, 'sci.electronics': 14, 'talk.politics.guns': 15, 'sci.space': 16, 'soc.religion.christian': 17, 'misc.forsale': 18, 'talk.religion.misc': 19}
id2classes = {'talk.politics.mideast': 0, 'rec.autos': 4, 'comp.sys.mac.hardware': 2, 'alt.atheism': 5, 'rec.sport.baseball': 1, 'comp.os.ms-windows.misc': 2, 'rec.sport.hockey': 1, 'sci.crypt': 3, 'sci.med': 3, 'talk.politics.misc': 0, 'rec.motorcycles': 4, 'comp.windows.x': 2, 'comp.graphics': 2, 'comp.sys.ibm.pc.hardware': 2, 'sci.electronics': 3, 'talk.politics.guns': 0, 'sci.space': 3, 'soc.religion.christian': 5, 'misc.forsale': 6, 'talk.religion.misc': 5}

number = range(50,150,50)
_per_segment = range(200,400,100)

overlap = {"side":"both", "number": 50}

f1_score = load_metric("f1"); precision = load_metric("precision"); recall = load_metric("recall")

def tokenize(batch):
    return bert_tokenizer(batch['text'], truncation=True, max_length=max_input_length,padding = True)

def compute_metrics(eval_pred):
    predictions, label = eval_pred
    predictions = np.argmax(predictions, axis = 1)
    return {"f1" :f1_score.compute(predictions = predictions, references = label, average = 'weighted'),
            "precision" : precision.compute(predictions = predictions, references = label, average = 'weighted'),
            "recall": recall.compute(predictions = predictions, references = label, average = 'weighted')}

def hp_space(trial):
  return {"per_device_train_batch_size": trial.suggest_discrete_uniform("per_device_train_batch_size", 8,32,8)
        ,"learning_rate": trial.suggest_float("learning_rate", 0.00001,0.00005, log = True)
        ,"num_train_epochs": trial.suggest_int("num_train_epochs",1,10)}

def bert_init():
  return BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 7)

def id_2_labels(x,adict):
  """converts labels/classes into a number using a dictionary"""
  return adict[x]

def _indexing(alist):
  id = 0; indexed = {}
  for x in alist:
    for y in x:
      id +=1
      indexed[id] = y
  return indexed
  

In [7]:
news_group_train = PrepareCorpus(train)
news_group_test = PrepareCorpus(test)

In [8]:
news_group_test = news_group_test._prep()       #Keys are labels, values are texts
news_group_train = news_group_train._prep()

In [9]:
#Full train and test data

train_list_of_strings = _indexing(news_group_train.values())
test_list_of_strings = _indexing(news_group_test.values())

train_corpus = AssessData(train_list_of_strings, news_group_train)
test_corpus = AssessData(test_list_of_strings, news_group_test)

In [10]:
train_index_to_label = train_corpus._index_to_label() 
test_index_to_label = test_corpus._index_to_label()

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
train_set = train_corpus._chunk(200, overlap=overlap)
train_index, train_text = zip(*train_set)

test_set = test_corpus._chunk(200, overlap=overlap)
test_index, test_text = zip(*test_set)

In [13]:
train_index = {a:x-1 for a,x in enumerate(train_index)}
train_text = {a:x for a,x in enumerate(train_text)}

In [14]:
test_index = {a:x-1 for a,x in enumerate(test_index)}
test_text = {a:x for a,x in enumerate(test_text)}

In [15]:
train_chunked_label = {i:id_2_labels(x,train_index_to_label) for i,x in train_index.items()}
test_chunked_label = {i:id_2_labels(x,train_index_to_label) for i,x in test_index.items()}

In [16]:
train_set = {"train" : {"text": train_text.values(), "doc_id": train_index.values(), "labels": train_chunked_label.values()}}
test_set =  {"test" : {"text": test_text.values(), 'doc_id': test_index.values(), 'labels': test_chunked_label.values()}}

In [17]:
import pandas as pd


a = pd.DataFrame(train_set['train'])
b = pd.DataFrame(test_set['test'])

In [18]:
b

,text,doc_id,labels
0,From: Sang.Shin@launchpad.unc.edu (SANG SHIN)\...,0,talk.politics.guns
1,gy\n Nntp-Posting-Host: lambada.oit.unc.edu\n ...,0,talk.politics.guns
2,ack down his post for his name)\n was talking ...,0,talk.politics.guns
3,"h in. Thus, each link is a possible locking p...",0,talk.politics.guns
4,ore effective than kryptonite cable locks (IMH...,0,talk.politics.guns
...,...,...,...
81855,From: Patrick Pearse Gallagher <pg23+@andrew.c...,7531,comp.os.ms-windows.misc
81856,NNTP-Posting-Host: po3.andrew.cmu.edu\n In-Re...,7531,comp.os.ms-windows.misc
81857,"t injury, but since he has\n >beenback (and be...",7531,comp.os.ms-windows.misc
81858,"I beg to differ, he had a couple 3 hit games ...",7531,comp.os.ms-windows.misc


In [19]:
#id2classes Maps 20 classes to 7 classes

a['labels'] = a['labels'].map(id2labels)
b['labels'] = b['labels'].map(id2labels)

In [20]:
b

,text,doc_id,labels
0,From: Sang.Shin@launchpad.unc.edu (SANG SHIN)\...,0,15
1,gy\n Nntp-Posting-Host: lambada.oit.unc.edu\n ...,0,15
2,ack down his post for his name)\n was talking ...,0,15
3,"h in. Thus, each link is a possible locking p...",0,15
4,ore effective than kryptonite cable locks (IMH...,0,15
...,...,...,...
81855,From: Patrick Pearse Gallagher <pg23+@andrew.c...,7531,5
81856,NNTP-Posting-Host: po3.andrew.cmu.edu\n In-Re...,7531,5
81857,"t injury, but since he has\n >beenback (and be...",7531,5
81858,"I beg to differ, he had a couple 3 hit games ...",7531,5


In [21]:
#Modify types

a['text'] = a['text'].astype(str)
a['doc_id'] = a['doc_id'].astype(int)
a['labels'] = a['labels'].astype(int)

b['text'] = b['text'].astype(str)
b['doc_id'] = b['doc_id'].astype(int)
b['labels'] = b['labels'].astype(int)

In [22]:
train = Dataset.from_pandas(a)
test = Dataset.from_pandas(b)

In [23]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [24]:
max_input_length = 300

def tokenize(batch):
    return bert_tokenizer(batch['text'], truncation=True, max_length=max_input_length,padding = True)

In [25]:
test = test.map(tokenize)

  0%|          | 0/81860 [00:00<?, ?ex/s]

In [26]:
#train = train.shuffle(seed = 30)
test = test.shuffle(seed = 30)

In [27]:
test = test.remove_columns('text')
test[0]

{'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'doc_id': 6752,
 'input_ids': [101,
  28549,
  6305,
  2923,
  1005,
  1055,
  5009,
  1998,
  2969,
  1011,
  2393,
  4475,
  2005,
  2423,
  5022,
  1012,
  2082,
  3454,
  2000,
  4652,
  9422,
  1024,
  1996,
  2120,
  4456,
  2820,
  5009,
  2000,
  9942,
  2008,
  9510,
  1006,
  3938,
  1011,
  3156,
  1007,
  1012,
  2023,
  5009,
  22106,
  2809,
  6827,
  3787,
  1997,
  1037,
  3112,
  102],
 'labels': 6,
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0]}

In [28]:
model_path = training_dir + '/results_5500_chunk'
model_path

'/content/drive/MyDrive/ML/Dissertation/training_cache/results_5500_chunk'

In [29]:
model = BertForSequenceClassification.from_pretrained(model_path)

In [30]:
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [31]:
from utils import RoBERT_Model, ToBERT_Model

In [32]:
R_Bert = RoBERT_Model(model, num_classes = model.classifier.out_features)
R_Bert_2 = RoBERT_Model_2(model, num_classes = model.classifier.out_features)


In [56]:
R_Bert.forward(np.array(test['input_ids']), np.array(test['attention_mask']), np.array(test['token_type_ids']), lengt = 251)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


TypeError: ignored